# Implementing the Mass Transport Approach for Discrete Choice Models


_Thomas Wiemann_

_TA Discussion \# 6_  
_ECON 31740_  

_Feburary 21, 2022_  

## Section 0: Preliminaries

This section loads the necessary dependencies. The mass transport approach is implemented in a custom package, ``MyMethods.jl``, which is readily available via https://github.com/thomaswiemann/MyMethods.jl. (See the ``README.md`` for installation instructions.) 

In [7]:
# Load custom package for basis functions. If not yet installed, run the following:
#    ``using Pkg; Pkg.add(PackageSpec(url="https://github.com/thomaswiemann/MyMethods.jl")))``.
using MyMethods

# Load additional dependencies
using Distributions, BenchmarkTools

## Section 1: A Most Simple Example

This section generates conditional choice probabilities of $J=3$ products in $T=100$ markets, with product-market specific characteristics $x_{jt}\in\mathbf{R}^2$. 

The true coefficients are 
$$
\alpha_0 = [1, 2, 3], \qquad \beta_0 = [1, -1].
$$

In [11]:
# Set dimensions of the data
T = 100;
J = 3;

# Generate example data and calculate conditional choice probabilities 
X = randn((T, J, 2))
mu = zeros((T, J))
for j in 1:(J )
    mu[:, j] = j .+ X[:, j, :] * [1, -1]
end
mu = exp.(mu)
CCP = mu ./ (1 .+ sum(mu, dims = 2));

We will attempt to recover these via the mass transport approach where we select the true distribution of the latent utility shocks ($\varepsilon \sim $T1EV).

In [12]:
# Define error distribution (T1EV)
Q = GeneralizedExtremeValue(-0.57721, 1, 0)

# Estimate and benchmark the mass transport problem
@btime mta_fit = myMTA(CCP, X, Q; S = 100);

  73.763 s (1589353 allocations: 97.26 MiB)


The estimated intercepts are reasonably aligned with $\alpha_0$,

In [13]:
# Check the intercept coefficients
α_hat = mta_fit.α

3-element Vector{Float64}:
 0.9945759918448884
 2.0090250216523846
 3.0027563177167167

Similarly, the slope coefficients are accurately estimated.

In [14]:
β_hat = mta_fit.β

2-element Vector{Float64}:
  0.9756573135461643
 -1.0063658039685408